1. selenium으로 토스 GoolePlay에 접속
2. 전체리뷰 보기 클릭
3. 리뷰창이 뜨면 최신순으로 정렬
4. 마우스 스크롤을 해서 오늘부터 2년전 날짜가 나올 때까지 스크롤
5. 데이터 수집
6. csv파일로 저장 → DB 저장
7. 수집코드 함수화
8. KB, 신한, 하나 리뷰도 저장

In [1]:
import time
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from general_dbio import *
print(dir())

['In', 'Out', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_ih', '_ii', '_iii', '_oh', 'create_engine', 'datetime', 'exit', 'get_ipython', 'open', 'pd', 'pymysql', 'quit', 'relativedelta', 'saveDB', 'time', 'timedelta']


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = Options()
options.add_experimental_option("detach", True)
options.add_argument('start-maximized')

# 사람인척하기위해서 개발자도구 network에서 user-agent추가, 언어 추가
options.add_argument('Chrome/134.0.0.0')
options.add_argument('lang=ko_KR')

# gpu 안쓰기
options.add_argument('--disable-gpu') 

In [3]:
idx = 1
columns = ['날짜', '닉네임', '리뷰내용']

end_date = datetime.today() - relativedelta(years=2) - timedelta(days=1)
end_date_text = end_date.strftime('%Y년 %m월 %d일')
print(end_date)
print(end_date_text)
print( datetime.today() >= datetime.strptime(end_date_text, '%Y년 %m월 %d일'))

2023-04-10 17:43:46.372429
2023년 04월 10일
True


In [9]:
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
    )

driver.get('https://play.google.com/store/apps/details?id=viva.republica.toss')

reviews_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "button[aria-label*='평점 및 리뷰 자세히 알아보기']"))
    )
reviews_btn.click()
order_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#sortBy_1"))
    )
order_btn.click()
recent_order = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[aria-label*="최신"]'))
    )
recent_order.click()

In [10]:
driver.find_element(By.CSS_SELECTOR, f'div.RHo1pe:nth-of-type(1) header span.bp9Aid')

<selenium.webdriver.remote.webelement.WebElement (session="8b0d27db0e05387fd3f11d5a25bdaa9f", element="f.95F1FCD22C7982473447E7F70EAD4C04.d.2C7D9B502DEC72C7818CA31D2590DD00.e.56")>

In [7]:
reviews_modal = driver.find_element(By.CSS_SELECTOR, '#yDmH0d > div.VfPpkd-Sx9Kwc.cC1eCc.UDxLd.PzCPDd.HQdjr.VfPpkd-Sx9Kwc-OWXEXe-FNFY6c > div.VfPpkd-wzTsW > div > div > div > div > div.fysCi.Vk3ZVd')

try: 
    while True:

        date = driver.find_element(By.CSS_SELECTOR, f'div.RHo1pe:nth-of-type({idx}) header span.bp9Aid').text
        nickname = driver.find_element(By.CSS_SELECTOR, f'div.RHo1pe:nth-of-type({idx}) header > div div.X5PpBb').text
        content = driver.find_element(By.CSS_SELECTOR, f'div.RHo1pe:nth-of-type({idx}) div.h3YV2d').text
        print(idx, date, nickname, content, end='\r')

        date_cast = datetime.strptime(date, '%Y년 %m월 %d일')
        if date_cast <= end_date:
            break

        temp = (date, nickname, content)
        df = pd.DataFrame([temp], columns=columns)
        saveDB(df, 'google_play_reviews', 'toss')
        time.sleep(1)

        driver.execute_script("arguments[0].scrollTop += 1000;", reviews_modal)  # 아래로 500px씩 스크롤
        time.sleep(1)

        idx += 1
except Exception as e:
    print(f'예외 발생: {type(e).__name__} - {e}')
finally:
    driver.quit()

KeyboardInterrupt: 

In [ ]:
driver.quit()